## Libraries etc.

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import networkx as nx
import subprocess

import os
import rdata
import dill as pickle

from collections import Counter, defaultdict
from datetime import datetime
from matplotlib import pyplot as plt
from itertools import combinations
#
from multiprocessing import Pool
import rdata
import warnings
from scipy.stats import entropy
warnings.filterwarnings("ignore")
#
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
# files
pickle_dir = './pickles/'
imputed_data_file = pickle_dir + 'imputed_data.xlsx'
pcaspace_dir = pickle_dir + '/pcaspaces/'
# parameters
jaccard_threshold=0.9
require_ft_present = True
min_n_tokens = 20
sampling_method = 'convenience'

## Part 2: analysis

In [3]:
df = pd.read_excel(imputed_data_file)

In [4]:
# sampling method
tot_wc = {l["doculect"] : (l['count']) for i,l in pd.read_csv('word_counts.csv').iterrows()}
doreco_metadata = pd.read_csv('doreco_languages_metadata_sep2023.csv')
doreco_metadata

,Language,Glottocode,iso-639-3,Family,fam_glottocode,Area,Creator,Latitude,Longitude,Archive,...,Audio license,DOI,Gloss,Extended speakers,Extended word tokens,Extended texts,Core speakers,core word tokens,Core texts,Years of recordings in core set
0,Anal,anal1239,anm,Sino-Tibetan,sino1245,Eurasia,"Ozerov, Pavel",24.05,94.28,ELAR,...,CC BY-NC,10.34847/nkl.0dbazp8m,none,12,13015,23,12,13015,23,2015-2016
1,Yali (Apahapsili),apah1238,na,Nuclear Trans New Guinea,nucl1709,Papunesia,"Riesberg, Sonja",-4.08,139.46,TLA,...,CC BY-NC-SA,10.34847/nkl.9d91nkq2,all,14,10191,11,10,7474,8,2013-2017
2,Arapaho,arap1274,arp,Algic,algi1248,North America,"Cowell, Andrew",43.39,-108.81,ELAR,...,CC BY,10.34847/nkl.36f5r1b6,all,8,10407,21,4,4746,11,2005
3,Baïnounk Gubëeher,bain1259,bab,Atlantic-Congo,atla1278,Africa,"Cobbinah, Alexander Yao",12.31,-16.06,ELAR,...,CC BY,10.34847/nkl.a332abw8,some,10,12425,18,9,11477,17,2009-2014
4,Beja,beja1238,bej,Afro-Asiatic,afro1255,Africa,"Vanhove, Martine",17.24,36.67,CorpoAfroas,...,CC BY-NC,10.34847/nkl.edd011t1,all,5,15439,58,5,15439,58,2003-2011
5,Bora,bora1263,boa,Boran,bora1262,South America,"Seifart, Frank",-2.00,-72.26,TLA,...,CC BY,10.34847/nkl.6eaf5laq,all,39,29578,38,6,8431,9,2004-2008
6,Cabécar,cabe1245,cjp,Chibchan,chib1249,North America,"Quesada, Juan Diego and Skopeteas, Stavros and...",9.67,-83.41,TLA,...,CC BY-NC-ND,10.34847/nkl.ebc4ra22,all,15,17961,115,10,10614,39,2011
7,Cashinahua,cash1254,cbs,Pano-Tacanan,pano1259,South America,"Reiter, Sabine",-9.72,-71.17,TLA,...,CC BY,10.34847/nkl.a8f9q2f1,some,3,10043,3,3,10043,3,2006
8,Dolgan,dolg1241,dlg,Turkic,turk1311,Eurasia,"Däbritz, Chris Lasse and Kudryakova, Nina and ...",71.11,94.29,HZSK,...,CC BY-NC,10.34847/nkl.f09eikq3,all,20,18966,25,6,8778,9,1972-2010
9,Evenki,even1259,evn,Tungusic,tung1282,Eurasia,"Kazakevich, Olga and Klyachko, Elena",61.97,94.69,Siberian Lang,...,CC BY,10.34847/nkl.5e0d27cu,all,23,8315,36,23,8315,36,2006-2016


In [5]:
area = defaultdict(lambda : [])
for f,c in Counter(doreco_metadata.Family).most_common():
    glottocodes = list(doreco_metadata[doreco_metadata.Family==f].Glottocode)
    found_glottocodes = [g for g in glottocodes if g in df.doculect.unique()]
    area_f = Counter(doreco_metadata[doreco_metadata.Family==f].Area).most_common(1)[0][0]
    #print(f,c,len(found_glottocodes),area_f)
    best_glottocode = max(found_glottocodes, key = tot_wc.get, default=None)
    # sample doculect with most words per genus
    if sampling_method in {'genus', 'genus-area'}:
        if best_glottocode != None: 
            area[area_f].append(best_glottocode)
    elif sampling_method == 'convenience':
        area[area_f].extend(found_glottocodes)
sample = []
for k,v in area.items():
    if sampling_method == 'genus-area' : 
        best_v = sorted(v, key = tot_wc.get)[-3:]
        # if method is genus-are, sample 3 doculects with most words per area
    else : best_v = v
    print(k,len(v),best_v)
    sample += best_v
print('>>>', sampling_method, len(sample), sample)
sampled_docs = set(sample)

Papunesia 9 ['nisv1234', 'nort2875', 'orko1234', 'port1286', 'sout2856', 'teop1238', 'vera1241', 'apah1238', 'komn1238']
Eurasia 16 ['anal1239', 'sadu1234', 'sumi1235', 'yong1270', 'lowe1385', 'nort2641', 'sout3282', 'dolg1241', 'urum1249', 'jeha1242', 'pnar1238', 'sanz1248', 'taba1259', 'even1259', 'kama1351', 'svan1243']
Africa 9 ['beja1238', 'goem1240', 'goro1270', 'bain1259', 'ruul1235', 'kaka1265', 'kark1256', 'nngg1234', 'tsim1256']
South America 7 ['movi1243', 'savo1255', 'yura1255', 'resi1247', 'trin1278', 'bora1263', 'cash1254']
North America 5 ['arap1274', 'cabe1245', 'hoch1243', 'texi1237', 'yuca1254']
Australia 1 ['ngal1292']
>>> convenience 47 ['nisv1234', 'nort2875', 'orko1234', 'port1286', 'sout2856', 'teop1238', 'vera1241', 'apah1238', 'komn1238', 'anal1239', 'sadu1234', 'sumi1235', 'yong1270', 'lowe1385', 'nort2641', 'sout3282', 'dolg1241', 'urum1249', 'jeha1242', 'pnar1238', 'sanz1248', 'taba1259', 'even1259', 'kama1351', 'svan1243', 'beja1238', 'goem1240', 'goro1270'

In [6]:
df_sub = df[df.doculect.isin(sampled_docs)]
print('n Sampled tokens: %d | n Original Tokens: %d' % (len(df_sub), len(df)))

n Sampled tokens: 146821 | n Original Tokens: 146821


In [10]:
field_anchors = []
fields = []
for doc in sampled_docs:
    print(doc)
    markers = df_sub[df_sub.doculect == doc].marker.unique()
    for m in markers:
        if require_ft_present: field = np.array((df_sub['imputed_%s' % doc] == m) & (df_sub['ft_present_%s' % doc]))
        else: field = np.array((df_sub['imputed_%s' % doc] == m))
        if sum(field) == 0: print(doc, m); continue
        fields.append(field)
        field_anchors.append((doc,m))
fields = np.array(fields)
print('n Fields: %d' % len(fields))

anal1239
port1286
sadu1234
sout3282
goem1240
dolg1241
beja1238
nort2641
arap1274
komn1238
movi1243
savo1255
kaka1265
yuca1254
bain1259
yura1255
resi1247
cabe1245
sumi1235
bora1263
texi1237
yong1270
ruul1235
kama1351
trin1278
nisv1234
nisv1234 nan
ngal1292
ngal1292 nan
even1259
urum1249
tsim1256
taba1259
hoch1243
goro1270
pnar1238
jeha1242
jeha1242 nan
orko1234
kark1256
sanz1248
lowe1385
nort2875
sout2856
vera1241
cash1254
teop1238
svan1243
apah1238
nngg1234
n Fields: 9535


In [11]:
def jaccard_similarity(si,sj):
    return (len(si & sj) / len(si | sj))

def get_merged_fields(fields, field_anchors, jaccard_threshold):
    sets = [set(np.where(fields[i])[0]) for i in range(fields.shape[0])]
    with Pool(12) as p:
        similarities = p.starmap(jaccard_similarity, map(lambda c : (sets[c[0]],sets[c[1]]), combinations(range(fields.shape[0]),2)))
    similar_field_pairs = {c for c,sim in zip(combinations(range(fields.shape[0]), 2), similarities) if sim >= jaccard_threshold}
    #
    G = nx.Graph()
    G.add_nodes_from(np.arange(len(fields)))
    G.add_edges_from(similar_field_pairs)
    #
    new_fields, new_anchors = [], []
    for c in nx.connected_components(G):
        similarities_c = [jaccard_similarity(sets[i],sets[j]) for i,j in combinations(c, 2)]
        new_field = np.zeros(fields.shape[1]).astype(bool)
        new_field[list(set.union(*[set(np.where(fields[ci])[0]) for ci in c]))] = True
        new_fields.append(new_field)
        new_anchors.append([field_anchors[ci] for ci in c])
        #
        if len(c) > 1: print('n Terms: %d | avg Jaccard sim: %.3f | min Jaccard sim: %.3f | n terms: %d' %
                             (len(c), np.mean(similarities_c), min(similarities_c), sum(new_field)))
    new_fields = np.array(new_fields)
    print('n New Fields: %d | n Old Fields: %d' % (len(new_fields), len(fields)))
    return new_fields, new_anchors

new_fields, new_anchors = get_merged_fields(fields, field_anchors, jaccard_threshold)

n Terms: 12 | avg Jaccard sim: 0.926 | min Jaccard sim: 0.839 | n terms: 820
n Terms: 17 | avg Jaccard sim: 0.883 | min Jaccard sim: 0.762 | n terms: 3083
n Terms: 11 | avg Jaccard sim: 0.904 | min Jaccard sim: 0.831 | n terms: 2253
n Terms: 17 | avg Jaccard sim: 0.922 | min Jaccard sim: 0.858 | n terms: 1021
n Terms: 13 | avg Jaccard sim: 0.947 | min Jaccard sim: 0.876 | n terms: 864
n Terms: 7 | avg Jaccard sim: 0.927 | min Jaccard sim: 0.864 | n terms: 201
n Terms: 5 | avg Jaccard sim: 0.903 | min Jaccard sim: 0.882 | n terms: 425
n Terms: 11 | avg Jaccard sim: 0.911 | min Jaccard sim: 0.829 | n terms: 541
n Terms: 4 | avg Jaccard sim: 0.925 | min Jaccard sim: 0.865 | n terms: 513
n Terms: 23 | avg Jaccard sim: 0.900 | min Jaccard sim: 0.793 | n terms: 1644
n Terms: 3 | avg Jaccard sim: 0.964 | min Jaccard sim: 0.947 | n terms: 430
n Terms: 14 | avg Jaccard sim: 0.940 | min Jaccard sim: 0.884 | n terms: 645
n Terms: 21 | avg Jaccard sim: 0.930 | min Jaccard sim: 0.842 | n terms: 419

In [12]:
def short_to_long(data_short):
    doculects = data_short.doculect.unique()
    index = 0
    colnames = []
    matrix = np.zeros((len(data_short),0)).astype(int)
    for doc in map(lambda k: 'imputed_' + k, doculects):
        ct = {ei : i for i,ei in enumerate(filter(lambda k : isinstance(k, str), data_short[doc].unique()))}
        matrix = np.hstack([matrix, np.zeros((len(data_short),len(ct)))])
        for ei,i in ct.items():
            colnames.append(doc + '|' + ei)
            matrix[:,index+i] = (np.array(data_short[doc] == ei).astype(int))#[int(e==ei) for e in data_short[doc]]
        index += len(ct)
    data_long = pd.DataFrame(matrix, columns=colnames)
    return data_long

In [15]:
def get_pc1(fn):
    identifier = os.getpid()
    subprocess.call(['Rscript', 'exppca.R', fn, str(identifier), 'logisticSVD', str(1)])
    model = rdata.conversion.convert(rdata.parser.parse_file('%s.rda' % identifier))['model']
    subprocess.call(['rm', '%s.rda' % identifier])
    return model

In [19]:
pickle.dump((new_anchors,new_fields), open('%s/fields.p' % pickle_dir, 'wb'))

In [20]:
def parallel_pca(x, boolean):
    doc, marker = x
    if os.path.isfile('%s/pcaspace_%s_%s.p' % (pcaspace_dir, doc, marker)): 
        return
    df_short = df_sub[boolean]
    df_long = short_to_long(df_short)
    good_columns = df_long.columns[(df_long.sum(0) > 0) & (df_long.sum(0) < len(df_long))]
    if len(good_columns) == 0: return
    fn = '%s/termspace_%s_%s.csv' % (pcaspace_dir, doc, marker)
    df_long[good_columns].to_csv(fn, index=False)
    #
    try: model = get_pc1(fn)
    except: model = None
    #
    subprocess.call(['rm', '%s/termspace_%s_%s.csv' % (pcaspace_dir, doc, marker)])
    if model != None: 
        pickle.dump(model, open('%s/pcaspace_%s_%s.p' % (pcaspace_dir, doc, marker), 'wb'))
        print(doc, marker, model['prop_deviance_expl'][0], datetime.now())

try: os.mkdir(pcaspace_dir)
except: pass
with Pool(8) as p:
    p.starmap(parallel_pca, zip(map(lambda k : k[0], new_anchors), new_fields))

Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted
Error: TridiagEigen: eigen decomposition failed
Execution halted
Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted
Error: TridiagEigen: eigen decomposition failed
Execution halted
Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted
Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted
Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <

ngal1292 h-bobon 0.2613770493916292 2024-03-25 14:08:49.257895


Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted


resi1247 ^ekaaka$ 0.1258335093578783 2024-03-25 14:08:53.986495
resi1247 htona 0.2968045076571354 2024-03-25 14:09:02.710268
ngal1292 -yenj 0.26683367172990013 2024-03-25 14:09:04.750987
resi1247 ^jipoji$ 0.3097988443717079 2024-03-25 14:09:05.346598
ruul1235 ruk 0.2753686378436554 2024-03-25 14:09:21.700682
ngal1292 -bu 0.30330695082720494 2024-03-25 14:09:22.464776
ngal1292 dje- 0.2664172576174655 2024-03-25 14:09:23.690231
ngal1292 ^nje 0.2377483781023807 2024-03-25 14:09:27.470077
yong1270 ^@tso$ 0.3510842103931292 2024-03-25 14:09:31.050917
ruul1235 oma 0.5275649745012491 2024-03-25 14:09:38.687974
tsim1256 ^gway$ 0.2532580020596823 2024-03-25 14:09:53.557832
tsim1256 she 0.2660914629127785 2024-03-25 14:09:57.645876
yong1270 ^le-tu-po-hw-dzo$ 0.19211237199440434 2024-03-25 14:09:57.649844
ngal1292 nginj$ 0.1759777680966934 2024-03-25 14:09:58.203470
tsim1256 anj 0.28557491740110186 2024-03-25 14:09:59.856333
ruul1235 a:ka$ 0.28041686702591684 2024-03-25 14:10:01.936239
yong1270 ^

Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted


...

Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted


sout2856 ^atua$ 0.25482718858717945 2024-03-25 15:01:09.997490
sout2856 ^runomser$ 0.22599746783189212 2024-03-25 15:01:12.008139


Error in fun(A, k, nu, nv, opts, mattype = "matrix") : 
  nrow(A) and ncol(A) should be at least 3
Calls: logisticSVD -> <Anonymous> -> <Anonymous> -> svds.matrix -> fun
Execution halted


sout2856 ^kemalik$ 0.22054541852143672 2024-03-25 15:01:15.369484
sout2856 ^nanwei$ 0.3431931217153814 2024-03-25 15:01:16.183598
sout2856 arik$ 0.3425708731518451 2024-03-25 15:01:17.250083
sout2856 ^apa$ 0.19894229347553327 2024-03-25 15:01:21.179159
sout2856 ^rufatlasik$ 0.40817154369577724 2024-03-25 15:01:22.221690
sout2856 slat 0.18019646981056647 2024-03-25 15:01:40.418394
sout2856 pa$ 0.1814903867323291 2024-03-25 15:01:44.604132
sout2856 sat 0.2765659388136694 2024-03-25 15:01:45.347315
sout2856 sel$ 0.22943832855322088 2024-03-25 15:01:46.412023
sout2856 pl 0.24517975092438993 2024-03-25 15:01:47.204529
sout2856 saiki$ 0.18918674476760144 2024-03-25 15:01:48.611128
sout2856 san$ 0.24019363027235885 2024-03-25 15:01:49.593586
sout2856 naor$ 0.2557413170051501 2024-03-25 15:01:50.591244
sout2856 ^nel$ 0.1479931273613747 2024-03-25 15:01:54.848723
sout2856 auswen$ 0.38039710661965254 2024-03-25 15:01:56.899140
sout2856 ^nafs 0.5680763799237447 2024-03-25 15:01:57.708542
sout2856

OSError: Cannot save file into a non-existent directory: 'pickles/pcaspaces/termspace_goro1270_^na'

## analysis

In [21]:
def get_ig(sorted_terms):
    H = entropy(list(Counter(sorted_terms).values()))
    max_ig = None, 0
    for i in range(1,len(sorted_terms)-1):
        A,B = sorted_terms[:i], sorted_terms[i:]
        Ha, Hb = entropy(list(Counter(A).values())), entropy(list(Counter(B).values()))
        IG = H - ((Ha * (i/len(sorted_terms))) + (Hb * (1-i/len(sorted_terms))))
        if IG > max_ig[1]: max_ig = i, IG
    return max_ig[1]

In [22]:
concreteness_file = '../metaphor/bert_exploration/13428_2013_403_MOESM1_ESM.xlsx'
conc_data = pd.read_excel(concreteness_file)
conc_dict = {r.Word : r['Conc.M'] for i,r in conc_data.iterrows()}
#
models = { 'knn@3' : KNeighborsClassifier(n_neighbors=3),
           'knn@5' : KNeighborsClassifier(n_neighbors=5),
           'SVC' : SVC(C=1) }

quantile_boundary_values = [0.10, 0.25, 0.5]
predictor_variables = ['std','range','iqr','log_fpm', 'concreteness'] + ['%d_balance' % (100*q) for q in quantile_boundary_values]

In [84]:
doreco_metadata

,Language,Glottocode,iso-639-3,Family,fam_glottocode,Area,Creator,Latitude,Longitude,Archive,...,Audio license,DOI,Gloss,Extended speakers,Extended word tokens,Extended texts,Core speakers,core word tokens,Core texts,Years of recordings in core set
0,Anal,anal1239,anm,Sino-Tibetan,sino1245,Eurasia,"Ozerov, Pavel",24.05,94.28,ELAR,...,CC BY-NC,10.34847/nkl.0dbazp8m,none,12,13015,23,12,13015,23,2015-2016
1,Yali (Apahapsili),apah1238,na,Nuclear Trans New Guinea,nucl1709,Papunesia,"Riesberg, Sonja",-4.08,139.46,TLA,...,CC BY-NC-SA,10.34847/nkl.9d91nkq2,all,14,10191,11,10,7474,8,2013-2017
2,Arapaho,arap1274,arp,Algic,algi1248,North America,"Cowell, Andrew",43.39,-108.81,ELAR,...,CC BY,10.34847/nkl.36f5r1b6,all,8,10407,21,4,4746,11,2005
3,Baïnounk Gubëeher,bain1259,bab,Atlantic-Congo,atla1278,Africa,"Cobbinah, Alexander Yao",12.31,-16.06,ELAR,...,CC BY,10.34847/nkl.a332abw8,some,10,12425,18,9,11477,17,2009-2014
4,Beja,beja1238,bej,Afro-Asiatic,afro1255,Africa,"Vanhove, Martine",17.24,36.67,CorpoAfroas,...,CC BY-NC,10.34847/nkl.edd011t1,all,5,15439,58,5,15439,58,2003-2011
5,Bora,bora1263,boa,Boran,bora1262,South America,"Seifart, Frank",-2.00,-72.26,TLA,...,CC BY,10.34847/nkl.6eaf5laq,all,39,29578,38,6,8431,9,2004-2008
6,Cabécar,cabe1245,cjp,Chibchan,chib1249,North America,"Quesada, Juan Diego and Skopeteas, Stavros and...",9.67,-83.41,TLA,...,CC BY-NC-ND,10.34847/nkl.ebc4ra22,all,15,17961,115,10,10614,39,2011
7,Cashinahua,cash1254,cbs,Pano-Tacanan,pano1259,South America,"Reiter, Sabine",-9.72,-71.17,TLA,...,CC BY,10.34847/nkl.a8f9q2f1,some,3,10043,3,3,10043,3,2006
8,Dolgan,dolg1241,dlg,Turkic,turk1311,Eurasia,"Däbritz, Chris Lasse and Kudryakova, Nina and ...",71.11,94.29,HZSK,...,CC BY-NC,10.34847/nkl.f09eikq3,all,20,18966,25,6,8778,9,1972-2010
9,Evenki,even1259,evn,Tungusic,tung1282,Eurasia,"Kazakevich, Olga and Klyachko, Elena",61.97,94.69,Siberian Lang,...,CC BY,10.34847/nkl.5e0d27cu,all,23,8315,36,23,8315,36,2006-2016


In [90]:
builder = []
for ai,(A, f) in enumerate(zip(new_anchors, new_fields)):
    df_ai = df_sub[f]
    translations = ['%s (%.1f%%)' % (k,100*v/len(df_ai)) for k,v in Counter(df_ai.term).most_common(3)]    
    #
    print(ai, len(df_ai), A[0], translations)
    #
    if next((False for t in df_ai.term.unique() if t in conc_dict),True):
        print('  *no concreteness scores'); continue
    field_concreteness = np.mean([conc_dict[k] for k,v in Counter(df_ai.term).items() if k in conc_dict for i in range(v)])
    #
    try: model = pickle.load(open(pcaspace_dir + 'pcaspace_' + A[0][0] + '_' + A[0][1] + '.p', 'rb'))
    except: continue
    good_docs = [k for k,v in Counter(df_ai.doculect).most_common() if v >= min_n_tokens] # min 20 tokens
    #
    space = model['A'][:,0]
    space = 2 * ((space - space.min()) / (space.max() - space.min())) - 1 # this places the PC space in [-1,1]
    quantile_boundaries = {i : (np.quantile(space, i), np.quantile(space, 1-i)) for i in quantile_boundary_values}
    #
    per_field_builder = []
    for doc in good_docs:
        observed = (df_ai.doculect == doc) & ~pd.isna(df_ai.marker)
        markers = df_ai[observed].marker
        space_doc = space[observed]
        if len(space_doc) < 1: continue
        if min(space_doc) > quantile_boundaries[0.25][0] or max(space_doc) < quantile_boundaries[0.25][1]: continue
        sorted_terms = list(zip(*sorted(zip(markers, space_doc), key = lambda k : k[1])))[0]
        row = {'doculect' : doc, 'field' : ai, 'concreteness' : field_concreteness }
        # PREDICTED VARIABLES:
        row['ig'] = get_ig(sorted_terms)
        #print('  ', doc, space_doc.reshape(-1,1).shape, markers.shape)
        for model_name, model in models.items():
            if len(set(markers)) == 1: row['colexified_%s' % model_name] = 1
            else:
                model.fit(space_doc.reshape(-1,1), markers)
                row['colexified_%s' % model_name] = int(model.predict([[-1]])[0] == model.predict([[+1]])[0])
        # PREDICTOR VARIABLES:
        row['std'] = np.std(space_doc)
        row['range'] = max(space_doc) - min(space_doc)
        row['iqr'] = np.quantile(space_doc, 0.75) - np.quantile(space_doc,0.25)
        row['log_fpm'] = np.log(1e6*len(markers)/tot_wc[doc])
        for q,(lo,hi) in quantile_boundaries.items():
            neg = sorted_terms[:sum(space_doc < lo)]
            pos = sorted_terms[-sum(space_doc >= hi):]
            row['%d_balance' % (100*q)] = 2*min(len(neg),len(pos))/(len(neg)+len(pos))
        per_field_builder.append(row)
    #

    #
    n_colex = sum(r['colexified_SVC'] for r in per_field_builder)
    n_dislex = len(per_field_builder)-n_colex
    if n_colex >= 0 and n_dislex >= 0: # Makes sure there are at least languages that colexify and ones that dislexify
        print('  sufficient variation (%d colex / %d dislex)' % (n_colex, n_dislex))
        mu_std = { m : (np.mean([r[m] for r in per_field_builder]),np.std([r[m] for r in per_field_builder]))
                  for m in predictor_variables }
        for i in range(len(per_field_builder)):
            for m in predictor_variables:
                per_field_builder[i]['offset_' + m] = ((per_field_builder[i][m]-mu_std[m][0])/(mu_std[m][1])) if mu_std[m][1] > 1e-9 else 0
        builder.extend(per_field_builder)
    else: print('  *insufficient variation (%d colex / %d dislex)' % (n_colex, n_dislex))
df_stats = pd.DataFrame(builder)

0 6388 ('anal1239', 'do') ['say (99.1%)', 'think (0.6%)', 'mean (0.1%)']
  sufficient variation (25 colex / 11 dislex)
1 1650 ('anal1239', 'da') ['say (97.4%)', 'mean (0.8%)', 'talk (0.7%)']
  sufficient variation (10 colex / 3 dislex)
2 98 ('anal1239', 'the') ['say (96.9%)', 'talk (2.0%)', 'speak (1.0%)']
  sufficient variation (1 colex / 0 dislex)
3 1258 ('anal1239', '^ju') ['go (96.4%)', 'move (1.5%)', 'get (1.3%)']
  sufficient variation (15 colex / 3 dislex)
4 2756 ('anal1239', 'cho') ['go (96.6%)', 'set (1.2%)', 'stay (0.8%)']
  sufficient variation (24 colex / 14 dislex)
5 1500 ('anal1239', 'vang') ['go (98.3%)', 'move (0.6%)', 'walk (0.5%)']
  sufficient variation (21 colex / 1 dislex)
6 2200 ('anal1239', '^ha') ['go (81.5%)', 'get (8.0%)', 'move (4.9%)']
  sufficient variation (2 colex / 25 dislex)
7 269 ('anal1239', '^vaj') ['go (95.2%)', 'get (1.9%)', 'stay (1.5%)']
  sufficient variation (0 colex / 3 dislex)
8 306 ('anal1239', 'ahung$') ['go (96.4%)', 'get (2.9%)', 'pass (0

In [91]:
print(len(df_stats.field.unique()), len(df_stats))

4679 33843


In [92]:
df_stats_old = df_stats.copy() # backup bc now you're zscoring

In [93]:
for i in predictor_variables + list(map(lambda k : 'offset_' + k, predictor_variables)):
    df_stats[i] = zscore(df_stats[i]) if len(set(df_stats[i])) > 1 else df_stats[i]

In [95]:
from scipy.stats import pearsonr, zscore, ttest_ind
#df_stats = df_stats_old.copy()
cont_dep = ['ig']
for i in predictor_variables + list(map(lambda k : 'offset_' + k, predictor_variables)):
    if len(set(df_stats[i])) == 1: 
        print(i,'insufficient variation');continue
    for d in cont_dep:
        p,r = pearsonr(zscore(df_stats[i]), df_stats[d])
        print(i,d,'%.2f (%.3f)' % (p,r), '<<<' if r < 0.05 else '')
        #sns.regplot(x=df_stats[i], y=df_stats[d])
        #plt.show()

std ig 0.67 (0.000) <<<
range ig 0.50 (0.000) <<<
iqr ig 0.53 (0.000) <<<
log_fpm ig -0.07 (0.000) <<<
concreteness ig 0.09 (0.000) <<<
10_balance ig 0.10 (0.000) <<<
25_balance ig 0.11 (0.000) <<<
50_balance ig 0.07 (0.000) <<<
offset_std ig 0.29 (0.000) <<<
offset_range ig 0.14 (0.000) <<<
offset_iqr ig 0.23 (0.000) <<<
offset_log_fpm ig -0.09 (0.000) <<<
offset_concreteness insufficient variation
offset_10_balance ig 0.09 (0.000) <<<
offset_25_balance ig 0.09 (0.000) <<<
offset_50_balance ig 0.08 (0.000) <<<


In [97]:
dfi = df_stats
lm = ols('ig ~ iqr + log_fpm', dfi).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                     ig   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     6932.
Date:                Mon, 25 Mar 2024   Prob (F-statistic):               0.00
Time:                        22:39:08   Log-Likelihood:                 6591.8
No. Observations:               33843   AIC:                        -1.318e+04
Df Residuals:                   33840   BIC:                        -1.315e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3443      0.001    318.047      0.0

In [99]:
import statsmodels.formula.api as smf
log_reg = smf.logit("colexified_SVC ~ iqr + log_fpm", data=df_stats).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.621212
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         colexified_SVC   No. Observations:                33843
Model:                          Logit   Df Residuals:                    33840
Method:                           MLE   Df Model:                            2
Date:                Mon, 25 Mar 2024   Pseudo R-squ.:                 0.07303
Time:                        23:39:01   Log-Likelihood:                -21024.
converged:                       True   LL-Null:                       -22680.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5066      0.012    -42.092      0.000      -0.530      -0.483
iqr           -0.7202      0.014    -51.771      0.000      -0.747      -0.693
log_fpm       -0.0799      0.012     -6.899      0.000      -0.103      -0.057
==============================================================================
"""

In [67]:
bin_dep = ['colexified_knn@3', 'colexified_knn@5', 'colexified_SVC']
for i in predictor_variables + list(map(lambda k : 'offset_' + k, predictor_variables)):
    if len(set(df_stats[i])) == 1: 
        print(i,'insufficient variation\n');continue
    for d in bin_dep:
        zed = zscore(df_stats[i])
        a,b = zed[df_stats[d] == 0], zed[df_stats[d] == 1]
        t,r = ttest_ind(a,b)
        print(i,d,'%.2f (%.3f)' % (t,r), '<<<' if r < 0.05 else '')
        #sns.regplot(x=df_stats[i], y=df_stats[d])
        #plt.show()
    print()

std colexified_knn@3 47.09 (0.000) <<<
std colexified_knn@5 51.74 (0.000) <<<
std colexified_SVC 77.95 (0.000) <<<

range colexified_knn@3 54.80 (0.000) <<<
range colexified_knn@5 51.19 (0.000) <<<
range colexified_SVC 104.04 (0.000) <<<

iqr colexified_knn@3 22.84 (0.000) <<<
iqr colexified_knn@5 29.07 (0.000) <<<
iqr colexified_SVC 35.44 (0.000) <<<

log_fpm colexified_knn@3 7.48 (0.000) <<<
log_fpm colexified_knn@5 9.96 (0.000) <<<
log_fpm colexified_SVC 5.75 (0.000) <<<

concreteness colexified_knn@3 5.34 (0.000) <<<
concreteness colexified_knn@5 5.56 (0.000) <<<
concreteness colexified_SVC 11.87 (0.000) <<<

10_balance colexified_knn@3 17.80 (0.000) <<<
10_balance colexified_knn@5 16.58 (0.000) <<<
10_balance colexified_SVC 28.84 (0.000) <<<

25_balance colexified_knn@3 11.79 (0.000) <<<
25_balance colexified_knn@5 12.31 (0.000) <<<
25_balance colexified_SVC 12.49 (0.000) <<<

50_balance colexified_knn@3 6.44 (0.000) <<<
50_balance colexified_knn@5 7.00 (0.000) <<<
50_balance cole